# Self Organized Maps

## Import Libraries

In [ ]:
from minisom import MiniSom
import matplotlib.pyplot as plt
import numpy as np

## Import catalog

In [ ]:
emu_matched_catalog = '/home/polaris/Lab_Astro/data/cross_match/cm5/EMU_0102-32_DES_VIKING_CATWISE_mag_pany056.fits'